### Load libraries & config

In [ ]:
from datetime import date, datetime
import girder_client as gc
import json
import os
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)

### Connect to Girder

In [ ]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [ ]:
activity_sets = girder_connection.get(
    "folder?parentType=collection&parentId={}".format(
        girder_connection.get(
            "collection?name=Volumes"
        )[0]["_id"]
    )
)

### Authorize Viewers to View their Users as Authorized in Admin Panel

In [ ]:
for a in activity_sets:
    for v in a['meta']['members']['viewers']:
        for u in a['meta']['members']['viewers'][v]:
            access = girder_connection.get("user/{}/access".format(u))
            if (v not in [user['id'] for user in access['users']]):
                for _id in [user['id'] for user in access['users']]:
                    access['users'].append({'id': v, 'level': 0})
                    girder_connection.put("user/{}/access".format(u),
                        data={"access":json.dumps(access)}
                    )
            try:
                responses = girder_connection.get(
                    "folder?parentType=user&parentId={}&name=Responses".format(
                        u
                    )
                )
                for responses_folder in responses:
                    try:
                        activity_set_responses = girder_connection.get(
                            "folder?parentType=folder&parentId={}&name={}".format(
                                responses_folder['_id'],
                                a['name']
                            )
                        )
                        for activity_set_response_folder in activity_set_responses: 
                            access = girder_connection.get(
                                "folder/{}/access".format(
                                    activity_set_response_folder['_id']
                                )
                            )
                            if v not in [uid["id"] for uid in access['users']]:
                                access['users'].append({'id': v, 'level': 0})
                                girder_connection.put(
                                    'folder/{}/access'.format(
                                        activity_set_response_folder['_id']
                                    ),
                                    data={
                                        "access":json.dumps(access),
                                        "recurse":True
                                    },
                                )
                    except:
                        ema_parent_responses = None
            except:
                responses = None

### Authorize Managers to View Users as Authorized in Admin Panel

In [ ]:
for a in activity_sets:
    for u in a['meta']['members']['users']:
        try:
            access = girder_connection.get(
                'user/{}/access'.format(u)
            )
            for m in a['meta']['members']['managers']:
                if m not in [uid['id'] for uid in access['users']]:
                    access['users'].append({'id': m, 'level': 0})
            girder_connection.put(
                'user/{}/access'.format(u),
                data={"access":json.dumps(access)}
            )
        except:
            access = {}

### Authorize Healthy Brain Network Data Access Group Data Access to All **EMA: Parent** Activity Data

In [ ]:
for user in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['users']:
    try:
        responses = girder_connection.get(
            "folder?parentType=user&parentId={}&name=Responses".format(
                user
            )
        )
        for responses_folder in responses:
            try:
                ema_parent_responses = girder_connection.get(
                    "folder?parentType=folder&parentId={}&name=EMA%3A%20Parent".format(
                        responses_folder['_id']
                    )
                )
                for ema_parent_response_folder in ema_parent_responses: 
                    access = girder_connection.get(
                        "folder/{}/access".format(
                            ema_parent_response_folder['_id']
                        )
                    )
                    if "5bab8fc80a76ac049becd5ec" not in [gid["id"] for gid in access['groups']]:
                        access['groups'].append({'id': '5bab8fc80a76ac049becd5ec', 'level': 0})
                        girder_connection.put(
                            'folder/{}/access'.format(
                                ema_parent_response_folder['_id']
                            ),
                            data={
                                "access":json.dumps(access),
                                "recurse":True
                            },
                        )
            except:
                ema_parent_responses = None
    except:
        responses = None

### Consolidate duplicated permissions

In [ ]:
for u in girder_connection.get('user'):
    access = girder_connection.get(
        'user/{}/access'.format(u['_id'])
    )
    new_access = {'groups': access['groups'], 'users': []}
    for au in access['users']:
        if au['level'] < 2:
            au['level'] = 0
        if au['id'] not in [uid['id'] for uid in new_access['users']]:
            new_access['users'].append(au)
    girder_connection.put(
        'user/{}/access'.format(u["_id"]),
        data={"access":json.dumps(new_access)}
    )